### **Importing Libraries**

In [ ]:
import math
import random
from collections import defaultdict
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import copy
from Card_and_Deck import *
from GameState import GameState
from Game_and_Player import *
import time

### **Sequential Halving**

In [ ]:
def simulate(game_state):
    """Simulates a random game from this state (play out the game to the end)."""
    game_state_new = copy.deepcopy(game_state)
    starting_player_index = game_state.current_player_index
    starting_player_name = game_state.players[starting_player_index].name

    while not game_state_new.is_terminal():
        legal_moves = game_state_new.get_legal_moves()
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        game_state_new.apply_move(move)

    rewards=game_state_new.tricks_won

    winner_name = max(rewards, key=rewards.get)

    winner_index = next(i for i, player in enumerate(game_state_new.players) if player.name == winner_name)


    return winner_index

In [ ]:
def sequential_halving_flat(state, budget):
    moves = state.get_legal_moves()
    # print(f"legal moves:{moves}")
    total_moves = len(moves)

    nplayouts = {(m.suit,m.rank): 0 for m in moves}
    nwins = {(m.suit,m.rank): 0 for m in moves}

    current_index = state.current_player_index

    while len(moves) > 1:
        sims_per_move = int(budget // (len(moves) * np.log2(total_moves)))
        if sims_per_move == 0:
            sims_per_move = 1

        for move in moves:
            for _ in range(sims_per_move):
                s = copy.deepcopy(state)
                s.apply_move(move)
                result = simulate(s)

                nplayouts[(move.suit,move.rank)] += 1
                if result == current_index:
                    nwins[(move.suit,move.rank)] += 1
                else:
                    nwins[(move.suit,move.rank)] += 0

        moves = bestHalf(state,moves, nwins, nplayouts)
        # print(f"half {moves}")

    return moves[0]


def bestHalf(state, moves, nwins, nplayouts):
    half = []
    notused = [True] * len(moves)

    for i in range(int(np.ceil(len(moves) / 2))):
        best = -1.0
        bestMove = moves[0]

        for index, m in enumerate(moves):
            if notused[index]:
                mu = nwins[(m.suit,m.rank)] / nplayouts[(m.suit,m.rank)] if nplayouts[(m.suit,m.rank)] > 0 else 0.0

                if mu > best:
                    best = mu
                    bestMove = m

        notused[moves.index(bestMove)] = False
        half.append(bestMove)

    return half


### **Determinizations**

In [ ]:
def uct_sampled_possible_worlds(game_state, num_samples=20, max_steps=100):



    best_actions = []
    for _ in range(num_samples):
        new_game_state = game_state.redistribute()
        best_action = sequential_halving_flat(new_game_state, max_steps)
        best_actions.append((best_action.suit,best_action.rank))

    best_move = max(best_actions, key=best_actions.count)
    best_move_card = Card(best_move[0], best_move[1])
    return best_move_card

### **Random Vs UCB Imperfect**

In [ ]:
def evaluate_monte_carlo(num_games=100, num_samples=20,max_steps=100,verbose=True):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)
        if verbose:
            print(f"Monte Carlo Player Index: {monte_carlo_player}")
        while not game_state.is_terminal():
            if game_state.current_player_index == monte_carlo_player:
                best_move = uct_sampled_possible_worlds(game_state, num_samples, max_steps)

            else:
                legal_moves1 = game_state.get_legal_moves()
                best_move = random.choice(legal_moves1)

            game_state.apply_move(best_move)


        rewards = game_state.tricks_won
        if verbose:
           print(rewards)

        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            if verbose:
                print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            if verbose:
                print(f"Monte Carlo player did not win: {winner_name} ")

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")



In [ ]:
start=time.time()
evaluate_monte_carlo(num_games=100, num_samples=20,max_steps=100,verbose=True)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Monte Carlo Player Index: 1
{'Alice': 3, 'Bob': 6, 'Charlie': 1}
Monte Carlo player won: Bob 
Monte Carlo Player Index: 2
{'Alice': 6, 'Bob': 4, 'Charlie': 0}
Monte Carlo player did not win: Alice 
Monte Carlo Player Index: 1
{'Alice': 2, 'Bob': 4, 'Charlie': 4}
Monte Carlo player won: Bob 
Monte Carlo Player Index: 0
{'Alice': 2, 'Bob': 6, 'Charlie': 2}
Monte Carlo player did not win: Bob 
Monte Carlo Player Index: 1
{'Alice': 3, 'Bob': 4, 'Charlie': 3}
Monte Carlo player won: Bob 
Monte Carlo Player Index: 2
{'Alice': 3, 'Bob': 3, 'Charlie': 4}
Monte Carlo player won: Charlie 
Monte Carlo Player Index: 1
{'Alice': 3, 'Bob': 4, 'Charlie': 3}
Monte Carlo player won: Bob 
Monte Carlo Player Index: 1
{'Alice': 5, 'Bob': 1, 'Charlie': 4}
Monte Carlo player did not win: Alice 
Monte Carlo Player Index: 1
{'Alice': 2, 'Bob': 5, 'Charlie': 3}
Monte Carlo player won: Bob 
Monte Carlo Player Index: 0
{'Alice': 5, 'Bob': 4, 'Charlie': 1}
Monte Carlo player won: Alice 
Monte Carlo Player Index: 

### **Rule Based Vs UCT Imperfect**

In [ ]:
def rule_based_move(game_state):
    legal_moves = game_state.get_legal_moves()
    return max(legal_moves, key=lambda card: card.rank)



def evaluate_monte_carlo_rule_based(num_games=100, num_samples=20,max_steps=100,verbose=True):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)
        while not game_state.is_terminal():
            if game_state.current_player_index == monte_carlo_player:
                best_move = uct_sampled_possible_worlds(game_state, num_samples, max_steps)

            else:
                best_move = rule_based_move(game_state)

            game_state.apply_move(best_move)


        rewards = game_state.tricks_won
        if verbose:
           print(rewards)

        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            if verbose:
                print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            if verbose:
                print(f"Monte Carlo player did not win: {winner_name} ")

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")



In [ ]:
start=time.time()
evaluate_monte_carlo_rule_based(num_games=100, num_samples=20,max_steps=100,verbose=True)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

{'Alice': 5, 'Bob': 0, 'Charlie': 5}
Monte Carlo player won: Alice 
{'Alice': 3, 'Bob': 1, 'Charlie': 6}
Monte Carlo player won: Charlie 
{'Alice': 4, 'Bob': 3, 'Charlie': 3}
Monte Carlo player won: Alice 
{'Alice': 4, 'Bob': 6, 'Charlie': 0}
Monte Carlo player did not win: Bob 
{'Alice': 0, 'Bob': 10, 'Charlie': 0}
Monte Carlo player did not win: Bob 
{'Alice': 1, 'Bob': 6, 'Charlie': 3}
Monte Carlo player did not win: Bob 
{'Alice': 3, 'Bob': 5, 'Charlie': 2}
Monte Carlo player did not win: Bob 
{'Alice': 5, 'Bob': 3, 'Charlie': 2}
Monte Carlo player did not win: Alice 
{'Alice': 7, 'Bob': 2, 'Charlie': 1}
Monte Carlo player won: Alice 
{'Alice': 4, 'Bob': 3, 'Charlie': 3}
Monte Carlo player did not win: Alice 
{'Alice': 4, 'Bob': 4, 'Charlie': 2}
Monte Carlo player won: Alice 
{'Alice': 4, 'Bob': 4, 'Charlie': 2}
Monte Carlo player did not win: Alice 
{'Alice': 4, 'Bob': 2, 'Charlie': 4}
Monte Carlo player won: Alice 
{'Alice': 0, 'Bob': 7, 'Charlie': 3}
Monte Carlo player did not w